# MLFlow Homework

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

Once you installed the package, run the command `mlflow --version` and check the output.

**What's the version that you have?**

Version 1.26.0

In [1]:
!mlflow --version

mlflow, version 1.26.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset.

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework) to preprocess the data

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

**How many files were saved to `OUTPUT_FOLDER`?**

❌ 1
❌ 3
✅ **4**
❌ 7

In [5]:
!python homework/preprocess_data.py --raw_data_path ../data --dest_path homework/output

## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

**How many parameters are automatically logged by MLflow?**

❌ 19
✅ 17
❌ 10
❌ 20


In [6]:
!python homework/train.py --data_path homework/output

2022/05/29 13:16:05 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

✅ `default-artifact-root`
❌ `serve-artifacts`
❌ `artifacts-only`
❌ `artifacts-destination`

Full command is `mlflow ui --backend-store-uri sqlite:///db-mlflow/taxi.db --default-artifact-root artifacts`

## Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

❌ 6.128
✅ 6.628
❌ 7.128
❌ 7.628

In [7]:
!python homework/hpo.py --data_path homework/output

100%|█████████| 50/50 [08:49<00:00, 10.58s/trial, best loss: 6.6284257482044735]


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

What is the test RMSE of the best model?

❌ 6.1
✅ 6.55
❌ 7.93
❌ 15.1

In [8]:
!python homework/register_model.py --data_path homework/output

2022/05/29 13:27:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Best model RMSE {'training_mse': 31.131050948533428, 'training_mae': 3.7900404294512096, 'training_r2_score': 0.7671660211119171, 'training_rmse': 5.579520673725784, 'training_score': 0.7671660211119171, 'valid_rmse': 6.6284257482044735, 'test_rmse': 6.547886185595423}
Registered model 'best-nyc-taxi-model' already exists. Creating a new version of this model...
2022/05/29 13:29:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best-nyc-taxi-model, version 4
Created version '4' of model 'best-nyc-taxi-model'.
